# Script 3 - Calculate index used in analyses

The index we are using to analyze the data was the Area Under the Curve (a.k.a SST Footprint). The mean area below the long term mean is calculated for each month for each EBUS/subregion.

| EBUS | Subregion | SST Threshold (Original) | SST Threshold (Modified) | 
| --- | --- | :---: | :---: | 
| California | Poleward | 11.88 | 11.88 | 
| California | Central | 13.07 | 13.07 | 
| California | Equatorward | 17.78 | 17.78 | 
| Humboldt | Poleward | 14.12 | 14.12 | 
| Humboldt | Central | 19.07 | 18.50 | 
| Humboldt | Equatorward | 18.27 | 18.30 | 
| Iberian/Canary | Poleward | 16.21 | 16.21 | 
| Iberian/Canary | Central | 19.26 | 19.08 | 
| Iberian/Canary | Equatorward | 22.57 | 22.12 | 
| Benguela | Poleward | 15.34 | 15.63 | 
| Benguela | Central | NA | 15.07 | 
| Benguela | Equatorward | 18.18 | 18.18 | 


In [ ]:
import numpy as np
import pandas as pd 
import xarray as xr
import matplotlib.pyplot as plt 
import matplotlib.patches as patches
import cartopy.feature as cfeature
import cartopy.crs as ccrs
import math
import warnings 
warnings.simplefilter('ignore') 
import seaborn as sns
import glob
import statsmodels.api as sm

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from calendar import month_abbr
from geopy import distance
from matplotlib.ticker import StrMethodFormatter
from datetime import datetime
import matplotlib.dates as mdates


### Loop through each EBUS, year, and month and count cells below mean region sst

In [ ]:
###Open data and calculate seasonal means
ddir = '../data/data_download/'
ebus = 'Iberian-Canary'
fns = glob.glob(ddir+ebus+'/*_MURR_*.nc')
ds = xr.open_mfdataset(fns)

#calculate seasonal index starting in december
ds_season = ds.resample(time = '1QS-DEC').mean()
ds

In [ ]:
###Calculate Seasonal Time Series and Detrend Seasonal Means
ddir = '../data/data_download/'
ebus_list = ['Iberian-Canary', 'Benguela', 'California', 'Humboldt']

ds_season_df = pd.DataFrame()

for ebus in ebus_list:
    
    fns = glob.glob(ddir+ebus+'/*_MURR_*.nc')
    ds = xr.open_mfdataset(fns)
    
    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)
        #sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (original subregions)
        #sst_upw = [17.337725, 14.049598, 13.024406] #long term, full region means
        reg = 3
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
        sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)
        #sst_upw = [14.118310928344727, 19.07120704650879, 18.2731876373291] #nearshore means (original subregions)
        #sst_upw = [14.972103, 19.707349, 19.989355] #long term, full region means
        reg = 3
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
        sst_upw = [22.11833381652832, 18.946455001831055, 16.212278366088867] #nearshore means (modified subregions with central Iberian changed)
        # sst_upw = [22.11833381652832, 19.076478958129883, 16.212278366088867] #nearshore means (modified subregions)
        #sst_upw = [22.574838638305664, 19.255435943603516, 16.212278366088867] #nearshore means (original subregions)
        #sst_upw = [23.338499, 20.541109, 17.248304] #long term, full region means
        reg = 3
    else: #Benguela
        lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
        lons = np.array([[13, 20],[10, 17],[8, 15]])
        sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)
        #sst_upw = [15.341087341308594, 18.181407928466797] #nearshore means (original subregions)
        #sst_upw = [17.249544, 19.3302917] #long term, full region means
        reg = 3
        
    #loop through the subregions of the ebus region
    for region in range(reg):

        #slice the data to the sub region
        ds_slice = ds.sel(lat=slice(lats[region,0],lats[region,1]),lon=slice(lons[region,0],lons[region,1]))

        #calculate seasonal index starting in december
        ds_season = ds_slice.resample(time = '1QS-DEC').mean()
        ds_temp = ds_season.groupby('time').mean(...).to_dataframe()
        ds_temp = ds_temp.reset_index()
        ds_temp['EBUS'] = ebus
        ds_temp['Subregion'] = region

        ds_season_df = pd.concat([ds_season_df, ds_temp])
        
#relabel regions
for ebus in ebus_list:
    if ebus in ['California', 'Iberian-Canary']:
        
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 0), 'Subregion'] = 'Equatorward'
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 1), 'Subregion'] = 'Central'
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 2), 'Subregion'] = 'Poleward'

    elif ebus == 'Bengula':
        
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 0), 'Subregion'] = 'Poleward'
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 1), 'Subregion'] = 'Equatorward'
        
    else:

        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 0), 'Subregion'] = 'Poleward'
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 1), 'Subregion'] = 'Central'
        ds_season_df.loc[(ds_season_df['EBUS'] == ebus) & (ds_season_df['Subregion'] == 2), 'Subregion'] = 'Equatorward'

#correct Benguela name
ds_season_df.loc[ds_season_df['EBUS'] == 'Bengula', 'EBUS'] = 'Benguela'

#extract out the month as a string
ds_season_df['time_str'] = ds_season_df['time'].dt.strftime('%m')

#calculate the seasonal means for each EBUS and Subregion
season_means = ds_season_df.groupby(['EBUS', 'Subregion', 'time_str']).mean().reset_index()
season_means = season_means.rename({'analysed_sst':'analysed_sst_mean'}, axis = 1)

#join the seasonal means with the dataframe
season_combined = ds_season_df.merge(season_means, on=['EBUS', 'Subregion', 'time_str'], how='left') 

#calculate the detrended values for each season
season_combined = season_combined.assign(analysed_sst_detrended = lambda x: x['analysed_sst'] - x['analysed_sst_mean'])

#save the data
season_combined.to_csv('../data/EBUS_Seasonal_Detrended_Means.csv', index = False)
season_combined

## **The cell below takes a long time to run**

In [ ]:
###Calculate the first index, the area under the cruve
#create empty dataframe
upwelling = pd.DataFrame(index=[0])

#directory path
ddir = '../data/data_download/'
#list of ebus regions
ebus_list = ['Iberian-Canary', 'Bengula', 'California', 'Humboldt']

#loop through different ebus regions
for ebus in ebus_list: 

    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (modified subregions)
        #sst_upw = [17.780235290527344, 13.065020561218262, 11.875925064086914] #nearshore means (original subregions)
        #sst_upw = [17.337725, 14.049598, 13.024406] #long term, full region means
        reg = 3
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-28,-17],[-17,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-72]]) 
        sst_upw = [14.118310928344727, 18.504968643188477, 18.300222396850586] #nearshore means (modified supregions)
        #sst_upw = [14.118310928344727, 19.07120704650879, 18.2731876373291] #nearshore means (original subregions)
        #sst_upw = [14.972103, 19.707349, 19.989355] #long term, full region means
        reg = 3
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,21.33],[21.33,33],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-8.75],[-14,-7]]) 
        sst_upw = [22.11833381652832, 18.946455001831055, 16.212278366088867] #nearshore means (modified subregions with central Iberian changed)
        # sst_upw = [22.11833381652832, 19.076478958129883, 16.212278366088867] #nearshore means (modified subregions)
        #sst_upw = [22.574838638305664, 19.255435943603516, 16.212278366088867] #nearshore means (original subregions)
        #sst_upw = [23.338499, 20.541109, 17.248304] #long term, full region means
        reg = 3
    else: #Benguela
        lats = np.array([[-34.8,-28.63],[-28.63, -22],[-22,-15]]) 
        lons = np.array([[13, 20],[10, 17],[8, 15]])
        sst_upw = [15.6278657913208, 15.074623107910156, 18.181407928466797] #nearshore means (modified subregions)
        #sst_upw = [15.341087341308594, 18.181407928466797] #nearshore means (original subregions)
        #sst_upw = [17.249544, 19.3302917] #long term, full region means
        reg = 3

    #open the specific ebus region
    fns = glob.glob(ddir+ebus+'/*_MURR_*.nc')
    ds = xr.open_mfdataset(fns)
    
    #resample data to be seasonal
    #ds = ds.resample(time = '1QS-DEC').mean()

    #loop through the subregions of the ebus region
    for region in range(reg):
    
        #loop through each year
        for i in ds.analysed_sst.time.values:
            #create an empty dictionary
            region_dict = {}
            
            #slice the data to the sub region
            ds_slice = ds.sel(lat=slice(lats[region,0],lats[region,1]),lon=slice(lons[region,0],lons[region,1]))
            #select out a single year and month
            tmp0 = ds_slice.analysed_sst.where(ds.time == i, drop=True)
            #filter the data so it's less than the mean annual sst for the region
            upwarea = tmp0.where(tmp0.values <= sst_upw[region])
            #convert it to a dataframe 
            upwarea_df = upwarea.to_dataframe()
            #count the number of cells to be the count of cells below the mean sst
            points = len(upwarea_df[upwarea_df['analysed_sst'].notnull()])
            #calculate total area (without nans)
            totals = xr.where(~np.isnan(tmp0.values), 1, 0).sum() 
            #convert it to a percent
            percent = np.around(points / totals * 100,2)

            #store each model parameter in the dictionary
            region_dict['EBUS'] = ebus
            region_dict['Region'] = str(region)
            region_dict['Year'] = str(int(tmp0.time.dt.year.values))
            region_dict['Month'] = str(int(tmp0.time.dt.month.values))
            region_dict['Points'] = str(points)
            region_dict['Percent'] = percent
            

            #add the dictionary as a row in the dataframe
            upwelling = upwelling.append(region_dict, ignore_index = True)
    print('Finished: ' + ebus)

#drop the first row (filler) and save file
upwelling = upwelling.drop(labels = 0, axis = 0)
upwelling.to_csv(ddir+ebus+'_upwelling_areas_seasonal_nearshore_Modified-Subregions.csv', index = False)
upwelling

In [ ]:
###Rejoin the files into one (originally split up cause accessing the google drive data for all four timed out)
ebus_list = ['Iberian-Canary', 'Benguela', 'California', 'Humboldt']

upwelling_joined = pd.DataFrame()

for ebus in ebus_list:
    temp = pd.read_csv(ddir+ebus+'_upwelling_areas_seasonal_nearshore_Modified-Subregions.csv')
    upwelling_joined = pd.concat([upwelling_joined, temp])
    
upwelling_joined.to_csv(ddir + 'upwelling_areas_seasonal_nearshore_Modified-Subregions.csv', index = False)

In [ ]:
###Clean up data
#read in upwelling results
ddir = '../data/data_download/'
upwelling = pd.read_csv(ddir+'upwelling_areas_seasonal_nearshore_Modified-Subregions.csv')
upwelling['Region_name'] = 'NA'
upwelling['EBUS_name'] = 'NA'
upwelling['Hemisphere'] = 'NA'

#clean up the columns of the data
for i in range(len(upwelling)):
    if upwelling['EBUS'][i] == 'Bengula':
        upwelling['EBUS_name'][i] = 'Benguela'
        if upwelling['Region'][i] == 0:
            upwelling['Region_name'][i] = 'South'
        else:
            upwelling['Region_name'][i] = 'North'
    else:
        upwelling['EBUS_name'][i] = upwelling['EBUS'][i]
        if upwelling['Region'][i] == 0:
            upwelling['Region_name'][i] = 'South'
        elif upwelling['Region'][i] == 1:
            upwelling['Region_name'][i] = 'Central'
        else:
            upwelling['Region_name'][i] = 'North'
            
for i in range(len(upwelling)):
    if (upwelling['EBUS_name'][i] == 'Benguela') | (upwelling['EBUS_name'][i] == 'Humboldt'):
        upwelling['Hemisphere'][i] = 'South'
    else:
        upwelling['Hemisphere'][i] = 'North'
        
for i in range(len(upwelling)):
    if upwelling['Hemisphere'][i] == 'North':
        if upwelling['Region_name'][i] == 'North':
            upwelling['Region_name'][i] = 'Poleward'
        elif upwelling['Region_name'][i] == 'Central':
            upwelling['Region_name'][i] = 'Central'
        else:
            upwelling['Region_name'][i] = 'Equatorward'
    else:
        if upwelling['Region_name'][i] == 'South':
            upwelling['Region_name'][i] = 'Poleward'
        elif upwelling['Region_name'][i] == 'Central':
            upwelling['Region_name'][i] = 'Central'
        else:
            upwelling['Region_name'][i] = 'Equatorward'
            
upwelling['Random_name'] = upwelling['EBUS_name'] + '-' + upwelling['Region_name']
upwelling_north = upwelling[upwelling['Hemisphere'] == 'North']
upwelling.to_csv('../data/upwellingfootprint_monthly_nearshore_updated_Modified-Subregions.csv', index = False)
upwelling

## ERA5 Charts

In [ ]:
#read in benguela data
beng_era5 = xr.open_dataset('../data/data_download/Benguela_ERA5.nc') #save data
beng_era5.close()
beng_df = beng_era5.to_dataframe().reset_index()
beng_df = beng_df[~beng_df['northward_wind_at_10_metres'].isnull()]
beng_df['EBUS'] = 'Benguela'

#read in california data
cali_era5 = xr.open_dataset('../data/data_download/California_ERA5.nc') #save data
cali_era5.close()
cali_df = cali_era5.to_dataframe().reset_index()
cali_df = cali_df[~cali_df['northward_wind_at_10_metres'].isnull()]
cali_df['EBUS'] = 'California'

#read in humboldt data
hum_era5 = xr.open_dataset('../data/data_download/Humboldt_ERA5.nc') #save data
hum_era5.close()
hum_df = hum_era5.to_dataframe().reset_index()
hum_df = hum_df[~hum_df['northward_wind_at_10_metres'].isnull()]
hum_df['EBUS'] = 'Humboldt'

#read in iberian data
iber_era5 = xr.open_dataset('../data/data_download/Iberian-Canary_ERA5.nc') #save data
iber_era5.close()
iber_df = iber_era5.to_dataframe().reset_index()
iber_df = iber_df[~iber_df['northward_wind_at_10_metres'].isnull()]
iber_df['EBUS'] = 'Iberian-Canary'

#join all the regions
frames = [beng_df, cali_df, hum_df, iber_df]
era5_joined = pd.concat(frames)
era5_joined

In [ ]:
#add in the subregions 
ebus_list = ['California', 'Humboldt', 'Iberian-Canary', 'Benguela']
era5_joined['Region'] = 'NA'

#loop through each ebus and add in the regions
for ebus in ebus_list:
    #select out coordinates based on EBUS
    if ebus == 'California':
        
        #list California region boundaries
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
                
        #add in regions
        era5_joined['Region'][(era5_joined['lat'] >= lats[0][0]) & (era5_joined['lat'] <= lats[0][1]) & (era5_joined['EBUS'] == ebus)] = 'South'
        era5_joined['Region'][(era5_joined['lat'] >= lats[1][0]) & (era5_joined['lat'] <= lats[1][1]) & (era5_joined['EBUS'] == ebus)] = 'Central'
        era5_joined['Region'][(era5_joined['lat'] >= lats[2][0]) & (era5_joined['lat'] <= lats[2][1]) & (era5_joined['EBUS'] == ebus)] = 'North'
    
    elif ebus == 'Humboldt':
        
        #list Humboldt region boundaries
        lats = np.array([[-42,-28],[-25,-17],[-15,-10]]) 
        
        #add in regions
        era5_joined['Region'][(era5_joined['lat'] >= lats[0][0]) & (era5_joined['lat'] <= lats[0][1]) & (era5_joined['EBUS'] == ebus)] = 'South'
        era5_joined['Region'][(era5_joined['lat'] >= lats[1][0]) & (era5_joined['lat'] <= lats[1][1]) & (era5_joined['EBUS'] == ebus)] = 'Central'
        era5_joined['Region'][(era5_joined['lat'] >= lats[2][0]) & (era5_joined['lat'] <= lats[2][1]) & (era5_joined['EBUS'] == ebus)] = 'North'
        
    elif ebus == 'Iberian-Canary':
        
        #list Iberian-Canary region boundaries
        lats = np.array([[15,20],[20,30],[37, 43.39]]) 
        
        #add in regions
        era5_joined['Region'][(era5_joined['lat'] >= lats[0][0]) & (era5_joined['lat'] <= lats[0][1]) & (era5_joined['EBUS'] == ebus)] = 'South'
        era5_joined['Region'][(era5_joined['lat'] >= lats[1][0]) & (era5_joined['lat'] <= lats[1][1]) & (era5_joined['EBUS'] == ebus)] = 'Central'
        era5_joined['Region'][(era5_joined['lat'] >= lats[2][0]) & (era5_joined['lat'] <= lats[2][1]) & (era5_joined['EBUS'] == ebus)] = 'North'

        
    else: #Benguela
        
        #list out Benguela region boundaries
        lats = np.array([[-34.8,-22],[-22,-15]]) 
     
        #add in regions
        era5_joined['Region'][(era5_joined['lat'] >= lats[0][0]) & (era5_joined['lat'] <= lats[0][1]) & (era5_joined['EBUS'] == ebus)] = 'South'
        era5_joined['Region'][(era5_joined['lat'] >= lats[1][0]) & (era5_joined['lat'] <= lats[1][1]) & (era5_joined['EBUS'] == ebus)] = 'North'
        
era5_joined = era5_joined.groupby(['time0', 'EBUS', 'Region']).mean().reset_index()
era5_joined = era5_joined[era5_joined.Region != 'NA']
era5_joined['sea_surface_temperature'] = era5_joined['sea_surface_temperature'] - 273.15
era5_joined = era5_joined.rename({'northward_wind_at_10_metres': 'ERA5_northward_wind_at_10_metres', 
                                  'northward_wind_stress': 'ERA5_northward_wind_stress',
                                 'sea_surface_temperature': 'ERA5_sea_surface_temperature'}, axis = 1)
era5_joined.to_csv('../data/ERA5_monthly_means.csv', index = False)
era5_joined

## ERA5 and MURR Climatology

### Calculate monthly means temps for MURR

In [ ]:
ddir = '../data/data_download/'

#create empty dataframe
temps = pd.DataFrame(index=[0])

#list of ebus regions
ebus_list = ['California', 'Humboldt', 'Iberian-Canary', 'Bengula']

#loop through different ebus regions
for ebus in ebus_list: 

    #select out coordinates based on EBUS
    if ebus == 'California':
        lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
        lons = np.array([[-124, -113],[-128, -120], [-129, -123]])
        sst_upw = [17.337725, 14.049598, 13.024406]
        reg = ['South', 'Central', 'North']
        ebus_name = ebus
    elif ebus == 'Humboldt':
        lats = np.array([[-42,-28],[-25,-17],[-15,-10]]) 
        lons = np.array([[-79,-69],[-76,-69],[-81,-75]]) 
        sst_upw = [14.972103, 19.707349, 19.989355]
        reg = ['South', 'Central', 'North']
        ebus_name = ebus
    elif ebus == 'Iberian-Canary':
        lats = np.array([[15,20],[20,30],[37, 43.39]]) 
        lons = np.array([[-21,-16],[-21,-9],[-14,-7]]) 
        sst_upw = [23.338499, 20.541109, 17.248304]
        reg = ['South', 'Central', 'North']
        ebus_name = ebus
    else: #Bengula
        lats = np.array([[-34.8,-22],[-22,-15]]) 
        lons = np.array([[10, 19],[8, 15]])
        sst_upw = [17.249544, 19.3302917]
        ebus_name = 'Benguela'
        reg = ['South', 'North']

    #open the specific ebus region
    fns = glob.glob(ddir+ebus+'/*.nc')
    ds = xr.open_mfdataset(fns)

    #loop through the subregions of the ebus region
    for index, region in enumerate(reg):
        
        #slice the data to the sub region
        ds_slice = ds.sel(lat=slice(lats[index,0],lats[index,1]),lon=slice(lons[index,0],lons[index,1]))
        ds_mean = ds_slice.analysed_sst.resample(time = 'M').mean(..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "Mean"}).reset_index()
        ds_10 = ds_slice.analysed_sst.resample(time = 'M').quantile(0.1, ..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "10thPerc"}).reset_index()
        ds_90 = ds_slice.analysed_sst.resample(time = 'M').quantile(0.9, ..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "90thPerc"}).reset_index()

        #join values
        df = ds_mean.merge(ds_10,on='time').merge(ds_90,on='time')
        df = df.drop(['quantile_x', 'quantile_y'], axis = 1)
        df['EBUS'] = ebus_name
        df['Region'] = reg[index]
        
        #join with existing dataframe
        temps = temps.append(df, ignore_index = True)
    
temps = temps.drop(labels = 0, axis = 0)
temps = temps.rename({'Mean':'MURR_Mean', '10thPerc':'MURR_10thPerc', '90thPerc':'MURR_90thPerc'}, axis = 1)
temps.to_csv('../data/MURR_monthly_means.csv', index = False)
temps

#### Offshore and Onshore MUR calculations

In [ ]:
ddir = '../data/data_download/Offshore_Onshorebands/'

onshore_offshore = pd.DataFrame(index = [0])

for shore in ['Nearshore', 'Offshore']:

    #create empty dataframe
    temps = pd.DataFrame(index=[0])

    #list of ebus regions
    ebus_list = ['California', 'Humboldt', 'Iberian-Canary', 'Bengula']

    #loop through different ebus regions
    for ebus in ebus_list: 

        #select out coordinates based on EBUS
        if ebus == 'California':
            lats = np.array([[28.5,34.5],[34.5,40.4],[40.4,46]]) 
            sst_upw = [17.337725, 14.049598, 13.024406]
            reg = ['South', 'Central', 'North']
            ebus_name = ebus
        elif ebus == 'Humboldt':
            lats = np.array([[-42,-28],[-25,-17],[-15,-10]]) 
            sst_upw = [14.972103, 19.707349, 19.989355]
            reg = ['South', 'Central', 'North']
            ebus_name = ebus
        elif ebus == 'Iberian-Canary':
            lats = np.array([[15,20],[20,30],[37, 43.39]]) 
            sst_upw = [23.338499, 20.541109, 17.248304]
            reg = ['South', 'Central', 'North']
            ebus_name = ebus
        else: #Bengula
            lats = np.array([[-34.8,-22],[-22,-15]]) 
            sst_upw = [17.249544, 19.3302917]
            ebus_name = 'Benguela'
            reg = ['South', 'North']

        #open the specific ebus region
        fns = glob.glob(ddir+ebus+'*'+shore+'*.nc')
        ds = xr.open_mfdataset(fns)
        ds.close()

        #loop through the subregions of the ebus region
        for index, region in enumerate(reg):

            #slice the data to the sub region
            ds_slice = ds.sel(lat=slice(lats[index,0],lats[index,1]))
            ds_mean = ds_slice.analysed_sst.resample(time = 'M').mean(..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "Mean"}).reset_index()
            ds_10 = ds_slice.analysed_sst.resample(time = 'M').quantile(0.1, ..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "10thPerc"}).reset_index()
            ds_90 = ds_slice.analysed_sst.resample(time = 'M').quantile(0.9, ..., skipna = True).to_dataframe().rename(columns={"analysed_sst": "90thPerc"}).reset_index()

            #join values
            df = ds_mean.merge(ds_10,on='time').merge(ds_90,on='time')
            df = df.drop(['quantile_x', 'quantile_y'], axis = 1)
            df['EBUS'] = ebus_name
            df['Region'] = reg[index]

            #join with existing dataframe
            temps = temps.append(df, ignore_index = True)

    temps = temps.drop(labels = 0, axis = 0)
    temps = temps.rename({'Mean':'MURR_Mean', '10thPerc':'MURR_10thPerc', '90thPerc':'MURR_90thPerc'}, axis = 1)
    
    temps['Shore'] = shore
    #temps.to_csv('./data/MURR_monthly_means.csv', index = False)
    
    onshore_offshore = pd.concat([onshore_offshore, temps])
    
onshore_offshore = onshore_offshore.drop(labels = 0, axis = 0)
onshore_offshore = onshore_offshore.reset_index().drop('index', axis = 1)
onshore_offshore.to_csv('../data/MURR_Nearshore_Offshore_monthly_means.csv', index = False)
onshore_offshore